# Analyze Elasto-Plastic with Dilatancy MC models

#### Author: Jonathan Moore

#### Purpose:
The purpose of this notebook is to run the incremental driver models with elasto-plastic  MC with dilatancy and then generate the associated plots.
Plots are:
* Quad Plot
* Error vs. Strain (Relative Error)
* 2-Norm error (standard eucliedean error) vs. Compute time
 


## Import Modules and make define functions

In [ ]:
# Standard imports
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import time
import re
from scipy.interpolate import interp1d
import pandas as pd


In [ ]:
sys.path.append(r"/mnt/data/the_deep/Geotech_Research/Critical_Soil_Models/pumat")

from lib.Load_Classes.Popular_Load_Class import PopularPath
from lib.Driver_Classes.Mod_Driver_Setup import DriverModelSetup
from lib.Driver_Classes.Mod_Driver_Model import DriverModel
from lib.general_functions.executing_runs import generate_batch_script, run_batch_script

In [ ]:
# Functions
def get_model_times(model_list, max_iters):
    model_times = []

    # Create a df with the number of loads as a header
    for model in model_list:
        i = 0
        while i < max_iters:
            start_time = time.time()
            model.run_model()
            end_time = time.time()

            elasped_time = end_time - start_time
            model_times.append(elasped_time)
            
            # Increment the counter
            i = i + 1

    return model_times

def calc_error(measured_val, true_val, error_type = "abs"):
    """ Calculates different types of error between a measured value and a true value"""

    match error_type:
        case "2norm":
            error = np.linalg.norm(measured_val - true_val)
        case "relative":
            # Calc the decimal percent error for each term
            error = np.abs( (measured_val - true_val) / true_val )
        case "abs":
            error = np.abs(measured_val - true_val)
        case "max_abs":
            error = np.max(np.abs(measured_val - true_val))
        case "mean_abs":
            error = np.mean(np.abs(measured_val - true_val))
        case _:
            raise ValueError("Error type not recognised")

    return error

def calc_k0(phi):
    """Calc the k0 value for a given phi using the relationship from (TODO: Add reference)"""
    return 1 - np.sin(phi)


def interp_data(interp_strain_vals, model_strain_data, model_value_data):
    """Function to interpolate data from a model to a new set of strain values"""
    
    model_strain_data = np.abs( model_strain_data )

    if not np.all(np.diff(interp_strain_vals) > 0):
        raise ValueError("Strain data must be in increasing order")
    
    # interpolate the values
    value = np.interp(interp_strain_vals, model_strain_data , model_value_data)

    # Return the interpolated values    
    return value


### Set the material and initial conditions for the models

In [ ]:
# Convert the friction angle from degrees to radians
# Setting the friction angle so it corresponds to a stress ratio of approx 1.0
phi = 25.4 * np.pi/180#25.375 * np.pi/180
stress_ratio = 6 * np.sin(phi)/(3 - np.sin(phi))

k0 = calc_k0(phi)

print(f"Stress ratio: {stress_ratio:.2}")
print(f"k0: {k0:.2}")

In [ ]:
# unit weight
# Calculating the unit weight to see the loads that occur at 

# Calc the unit weight assuming a porosity of 0.45
porosity = 0.45
particle_density = 2650.0 #[kg/m^3]
unit_weight = (1 - porosity) * particle_density * 9.81 #[N/m^3]


# Assume a depth of 1m and calc the vertical stress
vert_stress = unit_weight * 1

print(f"Unit weight: {unit_weight:.2f} N/m^3")
print(f"Vertical Stress at 1m depth: {vert_stress/1000:.2f} kN/m^2 (kPa)")

In [ ]:
# This needs to be in the order that the parameters should be in

initial_fric_angle = 25.375
initial_cohesion = 0
sub_properties = {
    "Shear Modulus (kPa)"    : 2500,
    "poisson ratio"          : 0.2,
    "peak cohesion"          : initial_cohesion,
    "residual cohesion"      : 0,
    "peak fric angle"        : initial_fric_angle,
    "residual fric angle"    : 0,
    "peak dilatancy"         : 15, 
    "residual dilatancy"     : 15,
    "softening shape factor" : 0,
    "integration flag"       : 0, # zero for substepping, 1 for Ortiz-Simo
    "Yield surface tolerance": 1e-8,
    "num integration iters"  : 1000,
    "Euler_DT_min"           : 1e-8 # minimum pseudo time step for sloan integration
}

# Set the ortiz properties to be the same as the newton properties except change the integration flag to be 1
ortiz_properties = sub_properties.copy()
ortiz_properties["integration flag"] = 1

### Set the folders for the models

In [ ]:
# Get the name of the current folder
current_working_dir = os.getcwd()

# Path to the folder that the Ortiz-Simo data should be stored in
Ortiz_folder = os.path.join(current_working_dir, "Ortiz_Simo")

# Path to the folder that the Newton Raphson data should be stored in
Newton_folder = os.path.join(current_working_dir, "Sloan_Abbo")

exe_path = r"/mnt/data/the_deep/Geotech_Research/Critical_Soil_Models/MohrCoulomb_StrainSoft/build/gfortran_E167FD2A985B468F/app/MCSS_incrementalDriver"


In [ ]:

OS_models = [
          DriverModel(os.path.join(current_working_dir, "Ortiz_Simo_500_ninc"), "Ortiz_MCSS", exe_path, "output.txt"),
          DriverModel(os.path.join(current_working_dir, "Ortiz_Simo_1000_ninc"), "Ortiz_MCSS", exe_path, "output.txt"),
          DriverModel(os.path.join(current_working_dir, "Ortiz_Simo_5000_ninc"), "Ortiz_MCSS", exe_path, "output.txt"),
]

Sub_models = [
          DriverModel(os.path.join(current_working_dir, "Sloan_Abbo_500_ninc"), "Sloan_Abbo", exe_path, "output.txt"),
          DriverModel(os.path.join(current_working_dir, "Sloan_Abbo_1000_ninc"), "Sloan_Abbo", exe_path, "output.txt"),
          DriverModel(os.path.join(current_working_dir, "Sloan_Abbo_5000_ninc"), "Sloan_Abbo", exe_path, "output.txt"),
]



In [ ]:
# Set the colors that will be used for plotting the model results later
colors = plt.cm.viridis(np.linspace(0, 1, len(OS_models)))

In [ ]:
for model in OS_models:
    model.setup.clear_folder()
    
for model in Sub_models:
    model.setup.clear_folder()

## Generate the model objects and make the input files for Incremental Driver

In [ ]:
trx_strain = "TriaxialE1"

nincs = [500, 1000, 5000] # Number of incremental steps to use

init_stress = -10 # Initial stress in kPa

# Set the stress and state parameters
stress = np.array([init_stress, k0 * init_stress, k0 * init_stress, 0, 0, 0])
state_params = {"cohesion": 0,
                "fric angle": initial_fric_angle * np.pi/180,
                "dilatancy angle": 0,
                "EspP_1": 0,
                "EspP_2": 0,
                "EspP_3": 0,
                "EspP_4": 0,
                "EspP_5": 0,
                "EspP_6": 0
}

load_params = {
        "ninc"   : None,
        "maxiter": 100,
        "dtime"  : 500_000,
        "every"  : 1,
        "ddstran_1": -0.4,
    }


# Create the substepping models
for inc, model in zip(nincs, Sub_models):
    # Set the number of strain increments
    temp_load_params = load_params.copy()
    temp_load_params["ninc"] = inc
    temp_load = PopularPath(trx_strain, temp_load_params)

    # Write the load parameters
    model.setup.write_initial_conditions_file(stress, state_params)
    # Clear the old loads
    model.setup.delete_all_loads()
    
    # Store the new loads
    model.setup.store_loads(temp_load)
    model.setup.write_loads()
    model.setup.write_parameters_file(sub_properties)

# Generate the OS models
for inc, model in zip(nincs, OS_models):
    # Set the number of strain increments
    temp_load_params = load_params.copy()
    temp_load_params["ninc"] = inc
    temp_load = PopularPath(trx_strain, temp_load_params)

    # Write the load parameters
    model.setup.write_initial_conditions_file(stress, state_params)
    
    # Clear the old loads
    model.setup.delete_all_loads()
    # Store the new load
    model.setup.store_loads(temp_load)
    model.setup.write_loads()
    model.setup.write_parameters_file(ortiz_properties)


## Run the models

In [ ]:
OS_times = []
max_iters = 10

OS_times = get_model_times(OS_models, max_iters)
print("")
Sub_times = get_model_times(Sub_models, max_iters)

#### Analyzing the computational time per model

In [ ]:
# Calculate the average time for each model
OS_avg_time = [None] * len(OS_models)
Sub_avg_time = [None] * len(Sub_models)

for i, model in enumerate(OS_models):
    OS_avg_time[i] = np.mean(OS_times[i*max_iters:(i+1)*max_iters])

for i, model in enumerate(Sub_models):
    Sub_avg_time[i] = np.mean(Sub_times[i*max_iters:(i+1)*max_iters])

In [ ]:
plt.plot(OS_times , color = colors[0], label="Ortiz-Simo")
plt.plot(Sub_times, color = colors[1], label="Sloan-Abbo")

avg_times = [max_iters/2, max_iters/2 + max_iters, max_iters/2 + 2 * max_iters]

# Plot the average times
plt.plot(avg_times, OS_avg_time, color = colors[0], marker = ".")
plt.plot(avg_times, Sub_avg_time, color = colors[1], marker = ".")

plt.xlabel("Iteration")
plt.ylabel("Time (s)")

plt.legend()
plt.show()

In [ ]:
# Incremental Driver is doing something strange at the start of the calculations so I'm removing that from the error analysis
val = 10

for model in OS_models:
    model.results.store_all()

for model in Sub_models:
    model.results.store_all()

In [ ]:
print(OS_models[-1].setup)

In [ ]:
# Setting the correct model to be the Substepping model with the highest iterations
correct_model = Sub_models[-1]

print(correct_model.setup)

## Calculate the model errors

In [ ]:
# Init lists to hold the errors
OS_q_error = []
OS_p_error = []
OS_eps_v = []


OS_errors = {
    "q_error": [],
    "p_error": [],
    "eps_v_error": [],
    "eps_q_error":[]
}


Sub_errors = {
    "q_error": [],
    "p_error": [],
    "eps_v_error": [],
    "eps_q_error":[]
}

In [ ]:

i=0

for model in OS_models:

    # Get the length of the 
    # Calc the (q) error

    # Get the strain values that the function should be evaluated at and take the abs value because interp function needs to be in increasing order
    correct_model_strain = np.abs( correct_model.results.strain_df["stran(1)"] )

    model_strain= np.abs( model.results.strain_df["stran(1)"] )
    
    # Interpolate the q invariant
    q_interp = interp_data(correct_model_strain, model_strain, model.results.get_q_invariant())
    
    OS_errors["q_error"].append(calc_error(q_interp, correct_model.results.get_q_invariant(), error_type="abs"))

    # Calc the p error
    # Interpolate the p invariant
    p_interp = interp_data(correct_model_strain, model_strain, model.results.get_mean_stress())
    
    OS_errors["p_error"].append(calc_error(p_interp, correct_model.results.get_mean_stress(), error_type="abs"))

    # eps_v error
    # Interpolate the volumetric strain
    eps_v_interp = interp_data(correct_model_strain, model_strain, model.results.get_volumetric_strain())
    OS_errors["eps_v_error"].append(calc_error(eps_v_interp, correct_model.results.get_volumetric_strain(), error_type="abs"))

    eps_q_interp = interp_data(correct_model_strain, model_strain, model.results.get_deviatoric_strain())
    OS_errors["eps_q_error"].append(calc_error(eps_q_interp, correct_model.results.get_deviatoric_strain(), error_type="abs"))

In [ ]:
for model in Sub_models:

    # Get the length of the 
    # Calc the (q) error

    # Get the strain values that the function should be evaluated at and take the abs value because interp function needs to be in increasing order
    correct_model_strain = np.abs( correct_model.results.strain_df["stran(1)"] )

    model_strain= np.abs( model.results.strain_df["stran(1)"] )
    
    # Interpolate the q invariant
    q_interp = interp_data(correct_model_strain, model_strain, model.results.get_q_invariant())
    
    Sub_errors["q_error"].append(calc_error(q_interp, correct_model.results.get_q_invariant(), error_type="abs"))

    # Calc the p error
    # Interpolate the p invariant
    p_interp = interp_data(correct_model_strain, model_strain, model.results.get_mean_stress())
    Sub_errors["p_error"].append(calc_error(p_interp, correct_model.results.get_mean_stress(), error_type="abs"))
    # eps_v error
    # Interpolate the volumetric strain
    eps_v_interp = interp_data(correct_model_strain, model_strain, model.results.get_volumetric_strain())
    
    Sub_errors["eps_v_error"].append(calc_error(eps_v_interp, correct_model.results.get_volumetric_strain(), error_type="abs"))
    
    eps_q_interp = interp_data(correct_model_strain, model_strain, model.results.get_deviatoric_strain())
    Sub_errors["eps_q_error"].append(calc_error(eps_q_interp, correct_model.results.get_deviatoric_strain(), error_type="abs"))



In [ ]:

OS_max_abs_errors = {
    "stress_error": [],
    "strain_error": []
}

for model in OS_models:
    # Interpolate the stress data
    stress_df = model.results.stress_df.copy()
    strain_df = model.results.strain_df.copy()

    stress_df_interp = pd.DataFrame()
    strain_df_interp = pd.DataFrame()
    
    x = np.linspace(0, 1, len(stress_df))
    new_x = np.linspace(0, 1, len(correct_model.results.stress_df))

    for col in stress_df.columns:
        interp_func = interp1d(x, stress_df[col], kind="linear")
        
        new_y = interp_func(new_x)
        stress_df_interp[col] = new_y

    for col in strain_df.columns:
        interp_func = interp1d(x, strain_df[col], kind="linear")

        new_y = interp_func(new_x)
        strain_df_interp[col] = new_y
    
    OS_max_abs_errors["stress_error"].append(calc_error(stress_df_interp[val:], correct_model.results.stress_df[val:], "max_abs"))
    
    OS_max_abs_errors["strain_error"].append(calc_error(strain_df_interp[val:], correct_model.results.strain_df[val:], "max_abs"))
    

In [ ]:

# Make a for the Substepping model
Sub_max_abs_errors = {
    "stress_error": [],
    "strain_error": []
}
for model in Sub_models:
    # Interpolate the stress data
    stress_df = model.results.stress_df.copy()
    strain_df = model.results.strain_df.copy()

    stress_df_interp = pd.DataFrame()
    strain_df_interp = pd.DataFrame()
    
    x = np.linspace(0, 1, len(stress_df))
    new_x = np.linspace(0, 1, len(correct_model.results.stress_df))

    for col in stress_df.columns:
        interp_func = interp1d(x, stress_df[col], kind="linear")
        
        new_y = interp_func(new_x)
        stress_df_interp[col] = new_y

    for col in strain_df.columns:
        interp_func = interp1d(x, strain_df[col], kind="linear")

        new_y = interp_func(new_x)
        strain_df_interp[col] = new_y
    
    Sub_max_abs_errors["stress_error"].append(calc_error(stress_df_interp[val:], correct_model.results.stress_df[val:], "max_abs"))
    
    Sub_max_abs_errors["strain_error"].append(calc_error(strain_df_interp[val:], correct_model.results.strain_df[val:], "max_abs"))
    

## Global error vs. computational time

In [ ]:
fig, axs = plt.subplots(nrows = 1, ncols = 2, figsize = (8, 3), dpi = 300)

error_names = ["stress_error", "strain_error"]


for i, error_name in enumerate(error_names):
    
    # Get the error
    OS_error = OS_max_abs_errors[error_name]
    Sub_error = Sub_max_abs_errors[error_name]

    color_index = 0
    # Plot the error
    axs[i].plot(OS_avg_time, OS_error, label = "OS", marker = ".", color = colors[color_index])
    axs[i].plot(Sub_avg_time, Sub_error, label = "Sloan-Abbo", marker = ".", linestyle = "dashed", color = colors[color_index])


# Format the first plot
axs[0].set_xlabel("Average Compute Time [s]")
axs[0].set_ylabel(" Max Abs. Error [kPa]")
axs[0].set_title("Stress Tensor Error")
axs[0].legend()

# Format the second plot
axs[1].set_title("Strain Tensor Error")
axs[1].set_xlabel("Average Compute Time [s]")
# axs[1].set_ylabel("Max Abs Strain Error [-]")


image_name = "Drained_Strain_Control_plastic_dilat_tensor_error.pdf"
mcss_latex_folder = "/home/jmoore/Documents/Master_Thesis/chapters/Constitutive_Modelling/mcss_images/plastic_dilat"
save_fig_path = os.path.join(mcss_latex_folder, image_name)
fig.savefig(save_fig_path, format = "pdf", bbox_inches = 'tight')
# plt.tight_layout()

plt.show()

## Absoloute Error vs Strain plots

In [ ]:
len(OS_errors["q_error"])

In [ ]:

fig, axs = plt.subplots(nrows = 1, ncols = 4, figsize = (15, 5), dpi = 300)

error_names = ["q_error", "p_error", "eps_v_error", "eps_q_error"]

for i, model in enumerate(OS_models):
            # Get the model name
    folder_name = os.path.basename(model.setup.folder_path)
    num_iters = re.search(r'\d+', folder_name).group()

    name = f"Ortiz-Simo  {num_iters} iters"

    # Loop over the error names
    for j, error_name in enumerate(error_names):
        
        error = OS_errors[error_name][i]
 
        # Plot the error versus the strain
        fig.axes[j].plot(correct_model_strain, error, label = name, color = colors[i])
        
        if j ==0:
           fig.axes[j].legend()

# Format the plot
fig.suptitle("Ortiz-Simo Error vs Strain")
axs[0].set_title("q Error")
axs[0].set_xlabel("$\\epsilon_{a}$ (-)")
axs[0].set_ylabel("Absolute Error")

axs[1].set_title("p Error")
axs[1].set_xlabel("$\\epsilon_{a}$ (-)")


axs[2].set_title("$\\epsilon_{v}$ Error")
axs[2].set_xlabel("$\\epsilon_{a}$ (-)")

axs[3].set_title("$\\epsilon_{q}$ Error")
axs[3].set_xlabel("$\\epsilon_{a}$ (-)")

# image_name = "Drained_Strain_Control_perf_plastic_OS_strain_error.pdf"
# mcss_latex_folder = "/home/jmoore/Documents/Master_Thesis/chapters/Constitutive_Modelling/mcss_images"
# save_fig_path = os.path.join(mcss_latex_folder, image_name)
# fig.savefig(save_fig_path, format = "pdf", bbox_inches = 'tight')

plt.tight_layout()
plt.show()

In [ ]:

fig, axs = plt.subplots(nrows = 1, ncols = 4, figsize = (15, 5), dpi = 300)

error_names = ["q_error", "p_error", "eps_v_error", "eps_q_error"]

for i, model in enumerate(Sub_models):
            # Get the model name
    folder_name = os.path.basename(model.setup.folder_path)
    num_iters = re.search(r'\d+', folder_name).group()

    name = f"Sloan-Abbo  {num_iters} iters"

    # Loop over the error names
    for j, error_name in enumerate(error_names):
        
        error = Sub_errors[error_name][i]
        
        # Plot the error versus the strain
        fig.axes[j].plot(correct_model_strain, error, label = name, color = colors[i])

        fig.axes[j].legend()

# Format the plot
fig.suptitle("Sloan-Abbo Absolute Error")
axs[0].set_title("q Error")
axs[0].set_xlabel("$\\epsilon_{a}$ (-)")
axs[0].set_ylabel("Absolute Error")

axs[1].set_title("p Error")
axs[1].set_xlabel("$\\epsilon_{a}$ (-)")
axs[0].set_ylabel("Absolute Error")

axs[2].set_title("Volumetric Strain Error")
axs[2].set_xlabel("$\\epsilon_{a}$ (-)")
axs[0].set_ylabel("Absolute Error")

plt.tight_layout()
# image_name = "Drained_Strain_Control_perf_plastic_substepping_error.pdf"
# mcss_latex_folder = "/home/jmoore/Documents/Master_Thesis/chapters/Constitutive_Modelling/mcss_images/perf_plastic"
# save_fig_path = os.path.join(mcss_latex_folder, image_name)
# fig.savefig(save_fig_path, format = "pdf", bbox_inches = 'tight')
plt.show()

In [ ]:

fig, axs = plt.subplots(nrows = 1, ncols = 4, figsize = (15, 5), dpi = 300)

error_names = ["p_error","q_error", "eps_v_error", "eps_q_error"]

for i, model in enumerate(OS_models):
            # Get the model name
    folder_name = os.path.basename(model.setup.folder_path)
    num_iters = re.search(r'\d+', folder_name).group()

    name = f"Ortiz-Simo  {num_iters} iters"

    # Loop over the error names
    for j, error_name in enumerate(error_names):
        
        error = OS_errors[error_name][i]
 
        # Plot the error versus the strain
        fig.axes[j].plot(correct_model_strain[val:], error[val:], label = name, color = colors[i])
        
        if j ==0:
           fig.axes[j].legend()

for i, model in enumerate(Sub_models):
            # Get the model name
    folder_name = os.path.basename(model.setup.folder_path)
    num_iters = re.search(r'\d+', folder_name).group()

    name = f"Sloan-Abbo  {num_iters} iters"

    # Loop over the error names
    for j, error_name in enumerate(error_names):
        
        error = Sub_errors[error_name][i]
        
        # Plot the error versus the strain
        fig.axes[j].plot(correct_model_strain[val:], error[val:], label = name, 
                         color = colors[i], linestyle = "dashed")

        if j ==0:
           fig.axes[j].legend()

# Format the plot
# fig.suptitle("Index Error vs Strain")
axs[0].set_title("p error")
axs[0].set_xlabel("$\\epsilon_{a}$ (-)")
axs[0].set_ylabel("Absolute Error")

axs[1].set_title("q error")
axs[1].set_xlabel("$\\epsilon_{a}$ (-)")

axs[2].set_title("$\\epsilon_{v}$ Error")
axs[2].set_xlabel("$\\epsilon_{a}$ (-)")

axs[3].set_title("$\\epsilon_{q}$ Error")
axs[3].set_xlabel("$\\epsilon_{a}$ (-)")

plt.tight_layout()
image_name = "Drained_Strain_Control_plastic_dilat_index_error.pdf"
mcss_latex_folder = "/home/jmoore/Documents/Master_Thesis/chapters/Constitutive_Modelling/mcss_images/plastic_dilat"
save_fig_path = os.path.join(mcss_latex_folder, image_name)
fig.savefig(save_fig_path, format = "pdf", bbox_inches = 'tight')

plt.show()

In [ ]:
# Generate a figure to plot the OS models

fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(8, 8), dpi=300)

for i, model in enumerate(OS_models):
    folder_name = os.path.basename(model.setup.folder_path)
    num_iters = re.search(r'\d+', folder_name).group()
    
    name = f"Ortiz-Simo  {num_iters} iters"
    model.results.quick_quad_plot(axial_strain_id="stran(1)", compression_pos=True, axs=axs, 
                                  color=colors[i], label=name, legend=[True, False, False, False], linestyle='-', linewidth=2)
# Add the Sloan Abbo model for comparison

for i, model in enumerate(Sub_models):
    
    folder_name = os.path.basename(model.setup.folder_path)
    num_iters = re.search(r'\d+', folder_name).group()
    
    name = f"Sloan-Abbo {num_iters} iters"
    model.results.quick_quad_plot(axial_strain_id="stran(1)", compression_pos=True, axs=axs, 
                                  color=colors[i], label=name, legend=[True, False, False, False], linestyle='dashed', linewidth=2)
    

image_name = "Drained_Strain_Control_plastic_dilat_quad.pdf"
mcss_latex_folder = "/home/jmoore/Documents/Master_Thesis/chapters/Constitutive_Modelling/mcss_images/plastic_dilat"
save_fig_path = os.path.join(mcss_latex_folder, image_name)
fig.savefig(save_fig_path, format = "pdf", bbox_inches = 'tight')
# plt.close()

plt.show()